In [1]:
import anndata as ad
import gc
import sys
from scipy.sparse import csc_matrix
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LinearRegression
import random
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import pickle
import matlab.engine
import heapq

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

cuda


In [13]:
train_mod1_file = 'phase2_data/predict_modality/openproblems_bmmc_cite_phase2_mod2/openproblems_bmmc_cite_phase2_mod2.censor_dataset.output_train_mod1.h5ad'
train_mod2_file = 'phase2_data/predict_modality/openproblems_bmmc_cite_phase2_mod2/openproblems_bmmc_cite_phase2_mod2.censor_dataset.output_train_mod2.h5ad'
# test_mod1_file = 'phase2_data/predict_modality/openproblems_bmmc_multiome_phase2_mod2/openproblems_bmmc_multiome_phase2_rna.censor_dataset.output_train_mod1.h5ad'
# test_mod1_file = 'phase2_data/predict_modality/openproblems_bmmc_multiome_phase2_mod2/openproblems_bmmc_multiome_phase2_rna.censor_dataset.output_train_mod2.h5ad'

In [14]:
input_train_mod1 = ad.read_h5ad(train_mod1_file)
input_train_mod2 = ad.read_h5ad(train_mod2_file)
# input_test_mod1 = ad.read_h5ad(test_mod1_file)
# input_test_mod2 = ad.read_h5ad(test_mod2_file)

In [15]:
print(input_train_mod1)
print(input_train_mod2)
# print(input_test_mod1)
# print(input_test_mod2)

AnnData object with n_obs × n_vars = 66175 × 134
    obs: 'batch'
    var: 'feature_types'
    uns: 'dataset_id', 'organism'
    layers: 'counts'
AnnData object with n_obs × n_vars = 66175 × 13953
    obs: 'batch'
    var: 'gene_ids', 'feature_types'
    uns: 'dataset_id', 'organism'
    layers: 'counts'


In [16]:
print(sorted(set(input_train_mod1.obs['batch'])))
print(sorted(set(input_train_mod2.obs['batch'])))

['s1d1', 's1d2', 's1d3', 's2d1', 's2d4', 's2d5', 's3d1', 's3d6', 's3d7']
['s1d1', 's1d2', 's1d3', 's2d1', 's2d4', 's2d5', 's3d1', 's3d6', 's3d7']


In [29]:
pro_s1d1 = input_train_mod1[input_train_mod1.obs["batch"] == "s1d1", :]
pro_s1d2 = input_train_mod1[input_train_mod1.obs["batch"] == "s1d2", :]
pro_s1d3 = input_train_mod1[input_train_mod1.obs["batch"] == "s1d3", :]
pro_s2d1 = input_train_mod1[input_train_mod1.obs["batch"] == "s2d1", :]
pro_s2d4 = input_train_mod1[input_train_mod1.obs["batch"] == "s2d4", :]
pro_s2d5 = input_train_mod1[input_train_mod1.obs["batch"] == "s2d5", :]

pro_s3d1 = input_train_mod1[input_train_mod1.obs["batch"] == "s3d1", :]
pro_s3d6 = input_train_mod1[input_train_mod1.obs["batch"] == "s3d6", :]
pro_s3d7 = input_train_mod1[input_train_mod1.obs["batch"] == "s3d7", :]

In [30]:
RNA_s1d1 = input_train_mod1[input_train_mod2.obs["batch"] == "s1d1", :]
RNA_s1d2 = input_train_mod1[input_train_mod2.obs["batch"] == "s1d2", :]
RNA_s1d3 = input_train_mod1[input_train_mod2.obs["batch"] == "s1d3", :]
RNA_s2d1 = input_train_mod1[input_train_mod2.obs["batch"] == "s2d1", :]
RNA_s2d4 = input_train_mod1[input_train_mod2.obs["batch"] == "s2d4", :]
RNA_s2d5 = input_train_mod1[input_train_mod2.obs["batch"] == "s2d5", :]

RNA_s3d1 = input_train_mod1[input_train_mod2.obs["batch"] == "s3d1", :]
RNA_s3d6 = input_train_mod1[input_train_mod2.obs["batch"] == "s3d6", :]
RNA_s3d7 = input_train_mod1[input_train_mod2.obs["batch"] == "s3d7", :]

In [31]:
pro_s1d1 = pro_s1d1.X.toarray()
pro_s1d2 = pro_s1d2.X.toarray()
pro_s1d3 = pro_s1d3.X.toarray()
pro_s2d1 = pro_s2d1.X.toarray()
pro_s2d4 = pro_s2d4.X.toarray()
pro_s2d5 = pro_s2d5.X.toarray()

pro_s3d1 = pro_s3d1.X.toarray()
pro_s3d6 = pro_s3d6.X.toarray()
pro_s3d7 = pro_s3d7.X.toarray()

In [32]:
RNA_s1d1 = RNA_s1d1.X.toarray()
RNA_s1d2 = RNA_s1d2.X.toarray()
RNA_s1d3 = RNA_s1d3.X.toarray()
RNA_s2d1 = RNA_s2d1.X.toarray()
RNA_s2d4 = RNA_s2d4.X.toarray()
RNA_s2d5 = RNA_s2d5.X.toarray()

RNA_s3d1 = RNA_s3d1.X.toarray()
RNA_s3d6 = RNA_s3d6.X.toarray()
RNA_s3d7 = RNA_s3d7.X.toarray()

In [34]:
pro_s1d1_mean = np.mean(pro_s1d1)
pro_s1d1_mean

0.8960458

In [36]:
train_input = [RNA_s1d1, RNA_s1d2, RNA_s1d3, RNA_s2d1, RNA_s2d4, RNA_s2d5]
train_output = [pro_s1d1, pro_s1d2, pro_s1d3, pro_s2d1, pro_s2d4, pro_s2d5]

test_input = [RNA_s3d1, RNA_s3d6, RNA_s3d7]
test_output = [pro_s3d1, pro_s3d6, pro_s3d7]

[[0.15437832 0.15437832 1.3004414  0.         0.40599617]
 [1.2411374  0.27233985 0.9930009  0.83153933 0.7073151 ]
 [0.         0.22376126 1.3568352  0.         0.22376126]
 [0.         0.11041228 0.92330563 0.3831878  2.3908489 ]
 [0.11822677 0.630471   0.91868454 0.4067957  0.7558843 ]]
[[-0.7416675  -0.7416675   0.40439558 -0.8960458  -0.49004963]
 [ 0.34509158 -0.623706    0.09695512 -0.06450647 -0.18873072]
 [-0.8960458  -0.67228454  0.46078944 -0.8960458  -0.67228454]
 [-0.8960458  -0.7856335   0.02725983 -0.51285803  1.4948031 ]
 [-0.77781904 -0.2655748   0.02263874 -0.4892501  -0.14016151]]


In [ ]:
# standardize based on each batch
for i in range(len(train_input)):
    train_input[i] = (train_input[i]-np.mean(train_input[i]))/np.std(train_input[i])

for i in range(len(train_output)):
    train_output[i] = (train_output[i]-np.mean(train_output[i]))/np.std(train_output[i])

for i in range(len(val_input)):
    val_input[i] = (val_input[i]-np.mean(val_input[i]))/np.std(val_input[i])

for i in range(len(val_output)):
    val_output[i] = (val_output[i]-np.mean(val_output[i]))/np.std(val_output[i])

In [ ]:
train_input = np.concatenate(train_input, axis=0)
val_input = np.concatenate(val_input, axis=0)

train_output = np.concatenate(train_output, axis=0)
val_output = np.concatenate(val_output, axis=0)

In [ ]:
train_input = torch.from_numpy(train_input)
train_output = torch.from_numpy(train_output)
val_input = torch.from_numpy(val_input)
val_output = torch.from_numpy(val_output)

In [ ]:
train_input = train_input.float()
train_output = train_output.float()
val_input = val_input.float()
val_output = val_output.float()

In [ ]:
num_epochs = 500
learning_rate = 0.01
latent_dim = 50
loss_fn = F.mse_loss
batch_size = 8

In [ ]:
train_ds = TensorDataset(train_input, train_output)
train_dl = DataLoader(train_ds, batch_size= batch_size, shuffle=True)

In [ ]:
input_feature = train_input.shape[1]
output_feature = train_output.shape[1]

In [37]:
# auto-encoder model
# base model
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.linear1 = nn.Linear(input_feature, input_feature//2)
        self.linear2 = nn.Linear(input_feature//2, input_feature//4)
        self.linear3 = nn.Linear(input_feature//4, input_feature//8)
        self.linear4 = nn.Linear(input_feature//8, input_feature//16)
        
        self.linear5 = nn.Linear(input_feature//16, output_feature)
        self.relu = nn.ReLU()
        self.leakyrelu = LeakyReLU(0.1)
        self.dropout = nn.Dropout(0.15)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.linear2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.linear3(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.linear4(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.linear5(x)
        output = self.relu(x)
        
        return output.float()

In [ ]:
def fit(num_epochs, model, loss_fn):
    val_min_loss = float('inf')
    counter = 0
    for epoch in range(num_epochs):
        for x,y in train_dl:
            model = model.train()
            pred = model(x)
            loss = loss_fn(pred)
            loss = torch.sqrt(loss)
#             loss = loss.float()
            loss.backward()
            opt.step()
            opt.zero_grad()

        if epoch % 100 == 0:
            loss = loss.cpu().detach().numpy()
            model = model.eval()
            
            train_pred = model(train_input)
            train_loss = loss_fn(train_pred, train_targets)
            train_loss = torch.sqrt(train_loss)
            train_loss = train_loss.cpu().detach().numpy()
            
            val_pred = model(val_input)
            val_loss = loss_fn(val_pred, val_output)
            val_loss = torch.sqrt(val_loss)
            val_loss = val_loss.cpu().detach().numpy()
            
#             test_pred = model(test_inputs)
#             test_loss = loss_fn(test_pred, test_targets)
#             test_loss = torch.sqrt(test_loss)
#             test_loss = test_loss.cpu().detach().numpy()

            print('Epoch ', epoch, 'Train_loss: ', train_loss, ' Validation_loss: ', val_loss)
    
    return train_pred, val_pred, test_pred

In [ ]:
model = Autoencoder()
opt = torch.optim.SGD(model.parameters(), lr=learning_rate)
fit(num_epochs, model, loss_fn)

In [ ]:
# model without relu in last layer
class Autoencoder_2(nn.Module):
    def __init__(self):
        super(Autoencoder_2, self).__init__()
        self.linear1 = nn.Linear(input_feature, input_feature//2)
        self.linear2 = nn.Linear(input_feature//2, input_feature//4)
        self.linear3 = nn.Linear(input_feature//4, input_feature//8)
        self.linear4 = nn.Linear(input_feature//8, input_feature//16)
        
        self.linear5 = nn.Linear(input_feature//16, output_feature)
        self.relu = nn.ReLU()
        self.leakyrelu = LeakyReLU(0.1)
        self.dropout = nn.Dropout(0.15)

    def forward(self, x):
        x = self.linear1(x)
        x = self.leakyrelu(x)
        x = self.dropout(x)
        
        x = self.linear2(x)
        x = self.leakyrelu(x)
        x = self.dropout(x)
        
        x = self.linear3(x)
        x = self.leakyrelu(x)
        x = self.dropout(x)
        
        x = self.linear4(x)
        x = self.leakyrelu(x)
        x = self.dropout(x)
        
        x = self.linear5(x)
        output = self.relu(x)
        
        return output.float()

In [ ]:
model = Autoencoder_2()
opt = torch.optim.SGD(model.parameters(), lr=learning_rate)
fit(num_epochs, model, loss_fn)

In [ ]:
# model with leakyrelu activation function
class Autoencoder_3(nn.Module):
    def __init__(self):
        super(Autoencoder_3, self).__init__()
        self.linear1 = nn.Linear(input_feature, input_feature//2)
        self.linear2 = nn.Linear(input_feature//2, input_feature//4)
        self.linear3 = nn.Linear(input_feature//4, input_feature//8)
        self.linear4 = nn.Linear(input_feature//8, input_feature//16)
        
        self.linear5 = nn.Linear(input_feature//16, output_feature)
        self.relu = nn.ReLU()
        self.leakyrelu = LeakyReLU(0.1)
        self.dropout = nn.Dropout(0.15)

    def forward(self, x):
        x = self.linear1(x)
        x = self.leakyrelu(x)
        x = self.dropout(x)
        
        x = self.linear2(x)
        x = self.leakyrelu(x)
        x = self.dropout(x)
        
        x = self.linear3(x)
        x = self.leakyrelu(x)
        x = self.dropout(x)
        
        x = self.linear4(x)
        x = self.leakyrelu(x)
        x = self.dropout(x)
        
        output = self.linear5(x)
        
        return output.float()

In [ ]:
model = Autoencoder_3()
opt = torch.optim.SGD(model.parameters(), lr=learning_rate)
fit(num_epochs, model, loss_fn)

In [ ]:
# resnet model
class Autoencoder_4(nn.Module):
    def __init__(self):
        super(Autoencoder_4, self).__init__()
        self.linear1 = nn.Linear(input_feature, input_feature//2)
        self.linear2 = nn.Linear(input_feature//2, input_feature//4)
        self.linear3 = nn.Linear(input_feature//2, input_feature//8)
        self.linear4 = nn.Linear(input_feature, input_feature//8)
        
        self.linear5 = nn.Linear(input_feature//4, input_feature//16)
        self.linear6 = nn.Linear(input_feature, input_feature//4)
        self.linear7 = nn.Linear(input_feature//16, output_feature)
        
        self.relu = nn.ReLU()
        self.leakyrelu = LeakyReLU(0.1)
        self.dropout = nn.Dropout(0.15)

    def forward(self, x):
        x1 = self.linear1(x)
        x1 = self.leakyrelu(x1)
        x1 = self.dropout(x1) # input_feature//2
        x1 = self.linear1(x1)
        x1 = self.leakyrelu(x1)
        x1 = self.dropout(x1) # input_feature//4
        
        x2 = self.linear6(x) # input_feature//4
        x2 = self.leakyreelu(x2)
        x2 = self.dropout(x2) # input_feature//4
        
        x3 = np.concatenate([x1, x2], axis=1) # input_feature//2
        x3 = self.leakyrelu(x3)
        x3 = self.dropout(x3)
        
        x4 = self.linear3(x3) # input_feature//8
        x5 = self.linear4(x) # input_feature//8
        x6 = np.concatenate([x4, x5], axis=1) # input_feature//4
        x6 = self.leakyrelu(x6)
        x6 = self.dropout(x6)
        
        x7 = self.linear5(x6) # input_feature//16
        x7 = self.leakyrelu(x7)
        x7 = self.dropout(x7)
        
        output = self.linear7(x7)
        
        return output.float()

In [ ]:
model = Autoencoder_4()
opt = torch.optim.SGD(model.parameters(), lr=learning_rate)
fit(num_epochs, model, loss_fn)

In [ ]:
torch.save(model.state_dict(), model_path)
model = Autoencoder_model()
model.load_state_dict(torch.load(model_path))
model.eval()
pred = model(test_inputs)
pred = pred.detach().numpy()

In [ ]:
plt.rcParams['figure.figsize'] = [40, 40]
figure, axis = plt.subplots(5, 5)

for i in range(25):
    y_true = test_y[:, i]
    y_pred = test_error[:, i] + test_y[:, i]
    max_ = max(max(y_true), max(y_pred))
    axis[i//5, i%5].scatter(y_true, y_pred)
    axis[i//5, i%5].plot([0, 5], [0, 5], 'k-')
    axis[i//5, i%5].set_xlim([0, max_])
    axis[i//5, i%5].set_ylim([0, max_])
    axis[i//5, i%5].set_title('Gene ' + ' ' + str(i) + ' mean square error ' + str(train_error_square_mean[i]))
    axis[i//5, i%5].set_xlabel('y_true')
    axis[i//5, i%5].set_ylabel('y_pred')
    axis[i//5, i%5].set_xlim((0, 5))
    axis[i//5, i%5].set_ylim((0, 5))
plt.show()